**This notebook implemented Question Answering, where based on the intent and entity extracted from user's input using trained Rasa NLU Intent Classifier and spaCy NER, the information/answer is extracted from the Neo4j knowledge graph using query.**

In [1]:
from neo4j import GraphDatabase
import spacy
from spacy.matcher import Matcher 
from spacy import displacy

### Import NER, Trained Rasa Intent Classifier, and Connect to Neo4j Driver

In [3]:
import spacy
ner = spacy.load("en_core_web_sm")

In [11]:
#load the trained model using Agent
from rasa.core.agent import Agent

agent = Agent.load(model_path='nlu-20231207-165856-simple-score.tar.gz')

C:\Users\yijin\anaconda3\envs\drlproject\lib\site-packages\future\standard_library\__init__.py:65: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
C:\Users\yijin\anaconda3\envs\drlproject\lib\site-packages\rasa\core\tracker_store.py:1044: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()


2023-12-07 17:09:46 [debug    ] graph.node.loading_component   clazz=NLUMessageConverter constructor=load kwargs={} node_name=nlu_message_converter
2023-12-07 17:09:46 [debug    ] graph.node.loading_component   clazz=FlowsProvider constructor=load kwargs={} node_name=flows_provider
2023-12-07 17:09:46 [debug    ] graph.node.loading_component   clazz=SpacyNLP constructor=load kwargs={} node_name=provide_SpacyNLP0
2023-12-07 17:09:47 [debug    ] graph.node.loading_component   clazz=SpacyNLP constructor=load kwargs={} node_name=run_SpacyNLP0
2023-12-07 17:09:47 [debug    ] graph.node.loading_component   clazz=SpacyTokenizer constructor=load kwargs={} node_name=run_SpacyTokenizer1
2023-12-07 17:09:47 [debug    ] graph.node.loading_component   clazz=SpacyEntityExtractor constructor=load kwargs={} node_name=run_SpacyEntityExtractor2
2023-12-07 17:09:47 [debug    ] graph.node.loading_component   clazz=SpacyFeaturizer constructor=load kwargs={} node_name=run_SpacyFeaturizer3
2023-12-07 17:09:4

C:\Users\yijin\anaconda3\envs\drlproject\lib\site-packages\rasa\utils\train_utils.py:525: UserWarning: constrain_similarities is set to `False`. It is recommended to set it to `True` when using cross-entropy loss.
  rasa.shared.utils.io.raise_warning(


2023-12-07 17:10:01 [debug    ] graph.node.loading_component   clazz=RegexMessageHandler constructor=load kwargs={} node_name=run_RegexMessageHandler


C:\Users\yijin\anaconda3\envs\drlproject\lib\site-packages\rasa\core\processor.py:112: UserWarning: The model metadata does not contain a value for the 'assistant_id' attribute. Check that 'config.yml' file contains a value for the 'assistant_id' key and re-train the model. Failure to do so will result in streaming events without a unique assistant identifier.
  rasa.shared.utils.io.raise_warning(


In [63]:
driver = GraphDatabase.driver("bolt://localhost:7687", 
                              auth=("neo4j", 
                                    "12345678"))

### Question Answering Design

In [231]:
# function to extract entities using spacy 
def extract_named_entity(text):
    # Process the text with spaCy NER
    doc = ner(text)

    # Extract named entities
    entities = [(ent.text, ent.label_) for ent in doc.ents]

    # Display the entities using spaCy's displacy
    displacy.render(doc, style="ent", jupyter=True)

    return entities

In [237]:
# function to extract intent and entities (using spacy and trained rasa)
async def get_intent_and_entities(input):

    # Extract intent
    result = await agent.parse_message(message_data = input)
    print(result)
    intent = result['intent']['name']
    #e = result['entities']['name']

    # Extract entities
    entities = extract_named_entity(input)

    return intent, entities

In [238]:
test = "I am curious, where did Ray Kurzweil worked at? Is it in America"
intent, entities = await get_intent_and_entities(test)
print(entities)

2023-12-08 10:33:03 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-08 10:33:03 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-08 10:33:03 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-08 10:33:03 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-08 10:33:03 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-08 10:33:03 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-08 10:33:03 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-08 10:33:03 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 

[('Ray Kurzweil', 'PERSON'), ('America', 'GPE')]


In [407]:
test = "Kurzweil Educational Systems"
intent, entities = await get_intent_and_entities(test)
print(entities)

2023-12-08 12:27:34 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-08 12:27:34 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-08 12:27:34 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-08 12:27:34 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-08 12:27:34 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-08 12:27:34 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-08 12:27:34 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-08 12:27:34 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 

[('Kurzweil Educational Systems', 'ORG')]


In [431]:
test = "Father of Ray Kurzweil"
intent, entities = await get_intent_and_entities(test)
print(entities)

2023-12-08 12:53:13 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-08 12:53:13 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-08 12:53:13 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-08 12:53:13 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-08 12:53:13 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-08 12:53:13 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-08 12:53:13 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-08 12:53:13 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 

[('Ray Kurzweil', 'PERSON')]


In [462]:
# function to handle who_question
def handle_who_question(entities, input):
    
    if not entities:
        return "Sorry, I couldn't identify the entity in the 'who' question."

    #entity_titles = [entity[0] for entity in entities if isinstance(entity, tuple)]
    entity_info = [(entity[0], entity[1]) for entity in entities if isinstance(entity, tuple)]
    entity_titles = [info[0] for info in entity_info]
    entity_labels = [info[1] for info in entity_info]

    if not entity_titles:
        return "Sorry, I couldn't identify the entity in the 'where' question."

    input_ques = input.lower()
    unique_responses = set()

    with driver.session() as session:

        if ((entity_labels[0] == 'ORG') and ('winner' in input_ques or 'award' in input_ques)):

            result = session.run(f"MATCH (p:Entity WHERE p.title = '{entity_titles[0]}')-[:`award received`|`winner`|`has received`]->(n) RETURN n.title AS n")

            for record in result:
                output = record["n"]

                if output is not None:
                    unique_responses.add(''.join(output))
                    
        elif (entity_labels[0] == 'ORG'):
            result = session.run(f"MATCH (p:Entity WHERE p.title = '{entity_titles[0]}')-[:`headquarters location`]->(n) RETURN n.title AS n")

            for record in result:
                output = record["n"]

                if output is not None:
                    unique_responses.add(''.join(output))

        elif (entity_labels[0] == 'PERSON'):
            if 'father' in input_ques or 'dad' in input_ques:
                result = session.run(f"MATCH (p:Entity WHERE p.title = '{entity_titles[0]}')-[:`father`]->(n) RETURN n.title AS n")

                for record in result:
                    output = record["n"]

                    if output is not None:
                        unique_responses.add(''.join(output))
                        
            if 'child' in input_ques or 'son' in input_ques or 'daughter' in input_ques:
                result = session.run(f"MATCH (p:Entity WHERE p.title = '{entity_titles[0]}')-[:`child`]->(n) RETURN n.title AS n")

                for record in result:
                    output = record["n"]

                    if output is not None:
                        unique_responses.add(''.join(output))

            if 'sibling' in input_ques or 'bro' in input_ques or 'sis' in input_ques:
                result = session.run(f"MATCH (p:Entity WHERE p.title = '{entity_titles[0]}')-[:`sibling`]->(n) RETURN n.title AS n")

                for record in result:
                    output = record["n"]

                    if output is not None:
                        unique_responses.add(''.join(output))

            if 'family' in input_ques:
                result = session.run(f"MATCH (p:Entity WHERE p.title = '{entity_titles[0]}')-[:`father`|`child`|`siblings`]->(n) RETURN n.title AS n")

                for record in result:
                    output = record["n"]

                    if output is not None:
                        unique_responses.add(''.join(output))

        if unique_responses:
            return "\n".join(unique_responses)
        else:
            return f"No information found about the locations related to {entity_titles[0]}."

In [463]:
# function to handle where_question
def handle_where_question(entities, input):
    
    if not entities:
        return "Sorry, I couldn't identify the entity in the 'where' question."

    #entity_titles = [entity[0] for entity in entities if isinstance(entity, tuple)]
    entity_info = [(entity[0], entity[1]) for entity in entities if isinstance(entity, tuple)]
    entity_titles = [info[0] for info in entity_info]
    entity_labels = [info[1] for info in entity_info]

    if not entity_titles:
        return "Sorry, I couldn't identify the entity in the 'where' question."

    input_ques = input.lower()
    unique_responses = set()

    with driver.session() as session:

        if (entity_labels[0] == 'PERSON'):

            result = session.run(f"MATCH (p:Entity WHERE p.title = '{entity_titles[0]}')-[:`educated at`]->(location) RETURN location.title AS locationTitle")

            for record in result:
                location_title = record["locationTitle"]

                if location_title is not None:
                   unique_responses.add(''.join(location_title))

        elif (entity_labels[0] == 'ORG'):
            result = session.run(f"MATCH (p:Entity WHERE p.title = '{entity_titles[0]}')-[:`headquarters location`]->(location) RETURN location.title AS locationTitle")


            for record in result:
                location_title = record["locationTitle"]

                if location_title is not None:
                   unique_responses.add(''.join(location_title))

        if unique_responses:
            return "\n".join(unique_responses)
        else:
            return f"No information found about the locations related to {entity_titles[0]}."

In [464]:
# function to handle when_question
def handle_when_question(entities, input):
    
    if not entities:
        return "Sorry, I couldn't identify the entity in the 'when' question."

    #entity_titles = [entity[0] for entity in entities if isinstance(entity, tuple)]
    entity_info = [(entity[0], entity[1]) for entity in entities if isinstance(entity, tuple)]
    entity_titles = [info[0] for info in entity_info]
    entity_labels = [info[1] for info in entity_info]

    if not entity_titles:
        return "Sorry, I couldn't identify the entity in the 'when' question."

    input_ques = input.lower()
    unique_responses = set()

    with driver.session() as session:

        if ((entity_labels[0] == 'PERSON') and ('born' in input_ques or 'birth' in input_ques)):

            result = session.run(f"MATCH (p:Entity WHERE p.title = '{entity_titles[0]}')-[:`date of birth`]->(n) RETURN n.title AS n")

            for record in result:
                output = record["n"]

                if output is not None:
                    unique_responses.add(''.join(output))

        elif ((entity_labels[0] == 'ORG') and ('invent' in input_ques or 'publish' in input_ques or 'publish' in input_ques)):
            result = session.run(f"MATCH (p:Entity WHERE p.title = '{entity_titles[0]}')-[:`publication date`]->(n) RETURN n.title AS n")

            for record in result:
                output = record["n"]

                if output is not None:
                    unique_responses.add(''.join(output))

        if unique_responses:
            return "\n".join(unique_responses)
        else:
            return f"No information found about the locations related to {entity_titles[0]}."

In [465]:
# function to handle what_question
def handle_what_question(entities, input):
    
    if not entities:
        return "Sorry, I couldn't identify the entity in the 'who' question."

    #entity_titles = [entity[0] for entity in entities if isinstance(entity, tuple)]
    entity_info = [(entity[0], entity[1]) for entity in entities if isinstance(entity, tuple)]
    entity_titles = [info[0] for info in entity_info]
    entity_labels = [info[1] for info in entity_info]

    if not entity_titles:
        return "Sorry, I couldn't identify the entity in the 'where' question."

    input_ques = input.lower()
    unique_responses = set()

    with driver.session() as session:

        if ((entity_labels[0] == 'PERSON') and ('winner' in input_ques or 'award' in input_ques)):

            result = session.run(f"MATCH (p:Entity WHERE p.title = '{entity_titles[0]}')-[:`award received`|`winner`|`has received`]->(n) RETURN n.title AS n")

            for record in result:
                output = record["n"]

                if output is not None:
                    unique_responses.add(''.join(output))

        elif ((entity_labels[0] == 'PERSON') and ('occupation' in input_ques or 'work as' in input_ques)):
            
            result = session.run(f"MATCH (p:Entity WHERE p.title = '{entity_titles[0]}')-[:`occupation`]->(n) RETURN n.title AS n")

            if result.peek() is None:
                response_messages.append(f"No information found about the location related to {entity_titles[0]}.")

            for record in result:
                output = record["n"]

                if output is not None:
                    unique_responses.add(''.join(output))

        elif ((entity_labels[0] == 'PERSON') and ('work' in input_ques or 'relate' in input_ques)):
            
            result = session.run(f"MATCH (p:Entity WHERE p.title = '{entity_titles[0]}')-[:`field of work`]->(n) RETURN n.title AS n")

            if result.peek() is None:
                response_messages.append(f"No information found about the location related to {entity_titles[0]}.")

            for record in result:
                output = record["n"]

                if output is not None:
                    unique_responses.add(''.join(output))

        if unique_responses:
            return "\n".join(unique_responses)
        else:
            return f"No information found about the locations related to {entity_titles[0]}."

In [466]:
# utility function that will take user's question as input and return answer
async def get_answer(input):
    
    input = input.replace("Raymond Kurzweil", "Ray Kurzweil")
    intent, entities = await get_intent_and_entities(input)
    
    if intent == "who_question":
        ans =  handle_who_question(entities, input)
    elif intent == "where_question":
        ans = handle_where_question(entities, input)
    elif intent == "when_question":
        ans = handle_when_question(entities, input)
    elif intent == "what_question":
        ans = handle_what_question(entities, input)
    else:
        ans = "Sorry, I don't understand what you're asking."

    print("\n\nQuestion: ", input)
    print("\nAnswer:\n", ans)

### Test the Question Answering Application

In [467]:
question = "Who is the winner of National Inventors Hall of Fame?"
await get_answer(question)

2023-12-08 13:59:51 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-08 13:59:51 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-08 13:59:51 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-08 13:59:51 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-08 13:59:51 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-08 13:59:51 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-08 13:59:51 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-08 13:59:51 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 



Question:  Who is the winner of National Inventors Hall of Fame?

Answer:
 Ray Kurzweil


In [468]:
question = "Who is the family of Ray Kurzweil?"
await get_answer(question)

2023-12-08 13:59:52 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-08 13:59:52 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-08 13:59:52 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-08 13:59:52 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-08 13:59:52 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-08 13:59:52 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-08 13:59:52 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-08 13:59:52 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 



Question:  Who is the family of Ray Kurzweil?

Answer:
 Fredric*


In [469]:
question = "Tell me the siblings of Ray Kurzweil."
await get_answer(question)

2023-12-08 13:59:54 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-08 13:59:54 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-08 13:59:54 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-08 13:59:54 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-08 13:59:54 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-08 13:59:54 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-08 13:59:54 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-08 13:59:54 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 



Question:  Tell me the siblings of Ray Kurzweil.

Answer:
 Enid*


In [473]:
question = "Where did Ray Kurzweil attended his college?"
await get_answer(question)

2023-12-08 14:00:17 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-08 14:00:17 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-08 14:00:17 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-08 14:00:17 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-08 14:00:17 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-08 14:00:17 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-08 14:00:17 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-08 14:00:17 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 



Question:  Where did Ray Kurzweil attended his college?

Answer:
 Rensselaer Polytechnic Institute
Stanford University
Yale University
Massachusetts Institute of Technology
Hofstra University
Berklee College of Music


In [474]:
question = "Where is Stanford University located at?"
await get_answer(question)

2023-12-08 14:00:20 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-08 14:00:20 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-08 14:00:20 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-08 14:00:20 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-08 14:00:20 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-08 14:00:21 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-08 14:00:21 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-08 14:00:21 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 



Question:  Where is Stanford University located at?

Answer:
 Palo Alto, California


In [475]:
question = "Where is Frank Rosenblatt educated at?"
await get_answer(question)

2023-12-08 14:00:23 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-08 14:00:23 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-08 14:00:23 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-08 14:00:23 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-08 14:00:23 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-08 14:00:23 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-08 14:00:23 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-08 14:00:23 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 



Question:  Where is Frank Rosenblatt educated at?

Answer:
 Cornell University


In [476]:
question = "What award did Ray Kurzweil awarded"
await get_answer(question)

2023-12-08 14:00:25 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-08 14:00:25 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-08 14:00:25 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-08 14:00:25 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-08 14:00:25 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-08 14:00:25 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-08 14:00:25 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-08 14:00:25 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 



Question:  What award did Ray Kurzweil awarded

Answer:
 20 honorary doctorates*
National Medal of Technology and Innovation
Lemelson–MIT Prize
National Inventors Hall of Fame
Dickson Prize


In [477]:
question = "What is the occupation of Ray Kurzweil"
await get_answer(question)

2023-12-08 14:00:27 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-08 14:00:27 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-08 14:00:27 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-08 14:00:28 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-08 14:00:28 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-08 14:00:28 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-08 14:00:28 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-08 14:00:28 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 



Question:  What is the occupation of Ray Kurzweil

Answer:
 Computer scientist


In [478]:
question = "What job is Theodore Modis worked as?"
await get_answer(question)

2023-12-08 14:00:31 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-08 14:00:31 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-08 14:00:31 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-08 14:00:31 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-08 14:00:31 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-08 14:00:31 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-08 14:00:31 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-08 14:00:31 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 



Question:  What job is Theodore Modis worked as?

Answer:
 Futurist


In [479]:
question = "What work is Daniel Dennett doing?"
await get_answer(question)

2023-12-08 14:00:33 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-08 14:00:33 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-08 14:00:33 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-08 14:00:33 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-08 14:00:33 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-08 14:00:33 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-08 14:00:33 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-08 14:00:33 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 



Question:  What work is Daniel Dennett doing?

Answer:
 Artificial intelligence


In [480]:
question = "What is the field of work of Ray Kurzweil?"
await get_answer(question)

2023-12-08 14:00:34 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-08 14:00:34 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-08 14:00:34 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-08 14:00:34 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-08 14:00:34 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-08 14:00:34 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-08 14:00:34 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-08 14:00:34 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 



Question:  What is the field of work of Ray Kurzweil?

Answer:
 Superintelligence


In [481]:
question = "When is Kurzweil Educational Systems invented?"
await get_answer(question)

2023-12-08 14:00:36 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-08 14:00:36 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-08 14:00:36 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-08 14:00:36 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-08 14:00:36 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-08 14:00:37 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-08 14:00:37 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-08 14:00:37 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 



Question:  When is Kurzweil Educational Systems invented?

Answer:
 1999


In [482]:
question = "When is Ray Kurzweil borned"
await get_answer(question)

2023-12-08 14:00:42 [debug    ] graph.node.running_component   clazz=NLUMessageConverter fn=convert_user_message node_name=nlu_message_converter
2023-12-08 14:00:42 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=provide node_name=provide_SpacyNLP0
2023-12-08 14:00:42 [debug    ] graph.node.running_component   clazz=SpacyNLP fn=process node_name=run_SpacyNLP0
2023-12-08 14:00:42 [debug    ] graph.node.running_component   clazz=SpacyTokenizer fn=process node_name=run_SpacyTokenizer1
2023-12-08 14:00:42 [debug    ] graph.node.running_component   clazz=SpacyEntityExtractor fn=process node_name=run_SpacyEntityExtractor2
2023-12-08 14:00:42 [debug    ] graph.node.running_component   clazz=SpacyFeaturizer fn=process node_name=run_SpacyFeaturizer3
2023-12-08 14:00:42 [debug    ] graph.node.running_component   clazz=CountVectorsFeaturizer fn=process node_name=run_CountVectorsFeaturizer4
2023-12-08 14:00:42 [debug    ] graph.node.running_component   clazz=DIETClassifier fn=process 



Question:  When is Ray Kurzweil borned

Answer:
 February 12, 1948*
